In [2]:
import os
import keras
from keras.api.models import load_model, Sequential
from keras.api.layers import Dense, Flatten, Dropout
from keras.api.optimizers import Adam
from keras.api.losses import SparseCategoricalCrossentropy
from keras import layers, activations, Model

In [3]:
# We can customize the batch size, image height, and image width
batch_size  = 32
img_height  = 224
img_width   = 224
channel     = 3

image_size              = (img_height, img_width)
image_size_with_channel = (img_height, img_width, channel)

# Constant value
CHECKPOINTS = "checkpoints"
LATEST_MODEL_NAME = "latest_model.keras"

In [4]:
(train_ds, val_ds) = keras.preprocessing.image_dataset_from_directory(
    directory="datasets/preprocessed_CLAHE",
    validation_split=0.2,
    subset="both",
    seed=69420,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 2426 files belonging to 3 classes.
Using 1941 files for training.
Using 485 files for validation.


I0000 00:00:1727033491.319568   37184 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727033491.607403   37184 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727033491.607494   37184 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727033491.611344   37184 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1727033491.611452   37184 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:0

In [5]:
image_augmentation = {
    "training": Sequential([
        layers.RandomRotation(factor=0.2),
        layers.RandomFlip()
    ])
}

In [6]:
resnet_model = keras.applications.ResNet50(
    include_top=False,
    classes=3,
    input_shape=(img_height, img_width, channel)
)

In [7]:
resnet_model.trainable = True
for layer in resnet_model.layers[:-4]:
    layer.trainable = False

In [8]:
inputs = keras.Input(shape=image_size_with_channel)
x = image_augmentation["training"](inputs)
x = keras.applications.resnet.preprocess_input(x)
x = resnet_model(x)
x = Flatten()(x)
x = Dense(256)(x)
x = Dropout(0.5)(x)
outputs = Dense(3, activation=activations.softmax)(x)
model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sequential          │ (None, 224, 224,  │          0 │ input_layer_1[0]… │
│ (Sequential)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item (GetItem)  │ (None, 224, 224)  │          0 │ sequential[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_1          │ (None, 224, 224)  │          0 │ sequential[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ get_item_2          │ (None, 224, 224)  │          0 │ sequential[0][0]  │
│ (GetItem)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stack (Stack)       │ (None, 224, 224,  │          0 │ get_item[0][0],   │
│                     │ 3)                │            │ get_item_1[0][0], │
│                     │                   │            │ get_item_2[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 224, 224,  │          0 │ stack[0][0]       │
│                     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ resnet50            │ (None, 7, 7,      │ 23,587,712 │ add[0][0]         │
│ (Functional)        │ 2048)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 100352)    │          0 │ resnet50[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │ 25,690,368 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 256)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 3)         │        771 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 49,278,851 (187.98 MB)

 Trainable params: 26,745,859 (102.03 MB)

 Non-trainable params: 22,532,992 (85.96 MB)

In [9]:
def get_early_stopping():
    return keras.callbacks.EarlyStopping(patience=5)

In [10]:
model.compile(
    optimizer=Adam(learning_rate=0.01),
    loss=SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [11]:
history = model.fit(train_ds, validation_data=val_ds, epochs=30, callbacks=[get_early_stopping()])

Epoch 1/30


2024-09-23 02:31:43.973725: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
W0000 00:00:1727033504.049771   39624 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033504.123607   39624 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033504.165210   39624 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033504.215507   39624 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033504.236127   39624 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033504.249575   39624 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033504.262132   39624 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033504.274081   39624 gpu_t

 1/61 ━━━━━━━━━━━━━━━━━━━━ 11:32 12s/step - accuracy: 0.2188 - loss: 5.7137

W0000 00:00:1727033506.624946   39626 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033506.629828   39626 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033506.635034   39626 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033506.640032   39626 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033506.644944   39626 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033506.648562   39626 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033506.654416   39626 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033506.658822   39626 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033506.664752   39626 gp

60/61 ━━━━━━━━━━━━━━━━━━━━ 0s 253ms/step - accuracy: 0.6183 - loss: 497.3600

...

61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 274ms/step - accuracy: 0.6189 - loss: 493.4402

W0000 00:00:1727033523.072426   39621 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033523.094743   39621 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033523.098645   39621 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033523.102716   39621 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033523.106544   39621 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033523.109420   39621 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033523.113184   39621 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033523.116682   39621 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033523.121269   39621 gp

61/61 ━━━━━━━━━━━━━━━━━━━━ 34s 369ms/step - accuracy: 0.6196 - loss: 489.6468 - val_accuracy: 0.7608 - val_loss: 57.1693
Epoch 2/30


W0000 00:00:1727033528.762805   39625 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033528.764853   39625 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033528.766933   39625 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033528.768985   39625 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033528.770916   39625 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033528.772899   39625 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033528.774826   39625 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033528.776591   39625 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1727033528.778285   39625 gp

61/61 ━━━━━━━━━━━━━━━━━━━━ 19s 309ms/step - accuracy: 0.6798 - loss: 44.3130 - val_accuracy: 0.7258 - val_loss: 58.7425
Epoch 3/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 19s 310ms/step - accuracy: 0.7668 - loss: 27.7733 - val_accuracy: 0.7423 - val_loss: 36.9697
Epoch 4/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 19s 309ms/step - accuracy: 0.7767 - loss: 28.3615 - val_accuracy: 0.7526 - val_loss: 34.2737
Epoch 5/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 19s 310ms/step - accuracy: 0.7580 - loss: 35.4945 - val_accuracy: 0.7711 - val_loss: 36.5643
Epoch 6/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 19s 313ms/step - accuracy: 0.7225 - loss: 53.0110 - val_accuracy: 0.7278 - val_loss: 55.8199
Epoch 7/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 19s 313ms/step - accuracy: 0.7495 - loss: 57.0450 - val_accuracy: 0.7814 - val_loss: 88.6861
Epoch 8/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 19s 311ms/step - accuracy: 0.7800 - loss: 55.3477 - val_accuracy: 0.8103 - val_loss: 67.3812
Epoch 9/30
61/61 ━━━━━━━━━━━━━━━━━━━━ 19s 312ms/step - accuracy: 0.7765 - loss: 56.8306 - val_